In [2]:
import requests

LEN_ALL_JOB_PAGE = 15
LEN_WFH_JOB_PAGE = 12
page = 1
url_get_first = requests.get(f'https://www.fatsecret.co.id/Default.aspx?pa=rs')
url_get_next = requests.get(f'https://www.fatsecret.co.id/Default.aspx?pa=rs&pg={page}')

In [3]:
from bs4 import BeautifulSoup

# Get Link Every Recipes

In [28]:
soup_all = BeautifulSoup(url_get_first.content, "html.parser")
containers = soup_all.find_all("form", id="ctl03")[0]
recipes = containers.find("table", class_="listtable searchResult")

In [73]:
for i in recipes.find_all("a", class_="prominent"):
    print(i.get("href"))

/resep/74332855-donat/Default.aspx
/resep/71694317-muffin-oat-pisang/Default.aspx
/resep/70672846-banana-bread-oats/Default.aspx
/resep/70171367-bilah-energi-oatmeal-cokelat/Default.aspx
/resep/67444129-kue-pisang/Default.aspx
/resep/67236434-mie-shirataki-goreng/Default.aspx
/resep/66836366-kue-oat-pisang/Default.aspx
/resep/66824678-gandum-dengan-buah/Default.aspx
/resep/66790743-saus-cokelat/Default.aspx
/resep/66245292-sandwich-telur-dadar/Default.aspx
/resep/65593665-banana-oats-coklat/Default.aspx
/resep/65370350-wafel/Default.aspx
/resep/65355205-tumis-kubis/Default.aspx
/resep/65329105-jus-hijau/Default.aspx
/resep/65203011-kue-oat-pisang/Default.aspx
/resep/65145041-kue-pisang-kukus/Default.aspx
/resep/64688892-ayam-dengan-jagung-dan-kacang/Default.aspx
/resep/64613063-sup-sosis-dengan-sayuran/Default.aspx
/resep/64589067-tahu-dihancurkan-dengan-telur-orak-arik/Default.aspx
/resep/64474557-sop-tulang-iga-sapi/Default.aspx


In [249]:
link_content =recipes.find_all("a", class_="prominent")[].get('href')
link_content

'/resep/71694317-muffin-oat-pisang/Default.aspx'

# Get Content

In [250]:
url_get_content = requests.get(f'https://www.fatsecret.co.id/{link_content}')

In [251]:
soup_content = BeautifulSoup(url_get_content.content, "html.parser")

## Porsi

In [252]:
serving = soup_content.find("div", id="servings")
serving = serving.find("div", class_="yield").text.strip()
serving

'5 porsi'

## Waktu Persiapan

In [253]:
cooking_time = soup_content.find("div", id="cooktime")
prep_time = cooking_time.find("div", class_="prepTime").text.strip()
cook_time = cooking_time.find("div", class_="cookTime").text.strip()
print(f"Prep: {prep_time}\nCook: {cook_time}")

Prep: 10 menit
Cook: 25 menit


## Jenis Makanan

In [254]:
meal_types = soup_content.find("div", id="mealtypes")
for i in meal_types.find_all("div", class_="tag"):
    print(i.text.strip())

Roti & Produk Panggang
Makanan Penutup
Makanan Ringan
Makan Pagi


## Nama Makanan

In [255]:
soup_content.find("h1", class_="fn").text.strip()

'Muffin Oat Pisang'

## Bahan

In [256]:
ingredients = []
for i in soup_content.find_all("li", class_="ingredient"):
    ingredients.append(i.text.strip())
ingredients

['1 sedang telur',
 '1 sdt ekstrak vanila',
 '154 gram pisang',
 '5 gram baking soda',
 '6 gram madu',
 '100 g quick cooking oatmeal',
 '50 g squeeze yoghurt']

## Intruksi

In [257]:
for i in soup_content.find_all("li", class_="instruction"):
    print(i.text.strip())

Masukan semua bahan pada blender.
Setelah tercampur, bagi adonan ke dalam cetakan.
Panggang dengan suhu 175°C selama 25 menit.


## Informasi Gizi

In [258]:
nutrition_fact = soup_content.find("div", class_="nutrition_facts international")

### Macro

In [259]:
macro_left = nutrition_fact.find_all("div", class_="nutrient black left")
macro_right = nutrition_fact.find_all("div", class_="nutrient black right tRight")
def get_macro(left, rigth):
    macro_dict = {}
    idx_rigth = 0
    for idx in range(len(left)):
        left_text = left[idx].text.strip()
        right_text = rigth[idx_rigth].text.strip()
        if left_text == '':
            pass
            idx_rigth -= 1
        else:
            macro_dict[left_text] = right_text
        idx_rigth += 1
    return macro_dict

macro = get_macro(macro_left, macro_right)

In [260]:
for key, value in macro.items():
    print(f"{key}: {value}")

Energi: 571 kj
Lemak: 2,98g
Protein: 4,02g
Karbohidrat: 23,62g


#### Sub Macro

In [264]:
right_texts = nutrition_fact.find_all("div", class_="nutrient right tRight")
sub_left_texts = nutrition_fact.find_all("div", class_="nutrient sub left")
left_texts = nutrition_fact.find_all("div", class_="nutrient left")

def get_sub_macro(left, sub_left, rigth):
    sub_macro_ = ['Lemak Jenuh',
        'Lemak Trans',
        'Lemak tak Jenuh Ganda',
        'Lemak tak Jenuh Tunggal',
        'Serat',
        'Gula',
        'Kolesterol',
        'Sodium',
        'Kalium']
    
    dict_sub_macro = {}
    idx_mg = 0
    idx_g = 0
    # check left text
    merge = [x.text.strip() for x in (sub_left_texts + left_texts)]
    missing_values = set(sub_macro_) - set(merge)
    for key in missing_values:
        dict_sub_macro[key] = '0g'
        
    for idx, right_text in enumerate(right_texts):
        portion = right_text.text.strip()
        if "kkal" in portion:
            dict_sub_macro["Kalori"] = portion
        elif "mg" in portion:
            dict_sub_macro[left[idx_mg].text.strip()] = portion
            idx_mg += 1
        else:
            dict_sub_macro[sub_left[idx_g].text.strip()] = portion
            idx_g += 1
    print(dict_sub_macro)
    return dict_sub_macro

sub_macro = get_sub_macro(left_texts, sub_left_texts, right_texts)

{'Kalori': '136 kkal', 'Lemak Jenuh': '0,593g', 'Lemak Trans': '0g', 'Lemak tak Jenuh Ganda': '0,714g', 'Lemak tak Jenuh Tunggal': '1,203g', 'Kolesterol': '37mg', 'Serat': '2,5g', 'Gula': '5,93g', 'Sodium': '295mg', 'Kalium': '138mg'}


# Get All Information

In [1]:
def get_macro(left, rigth):
    macro_dict = {}
    idx_rigth = 0
    for idx in range(len(left)):
        left_text = left[idx].text.strip()
        right_text = rigth[idx_rigth].text.strip()
        if left_text == '':
            pass
            idx_rigth -= 1
        else:
            macro_dict[left_text] = right_text
        idx_rigth += 1
    return macro_dict

In [2]:
def get_sub_macro(left, sub_left, rigth):
    sub_macro_ = ['Lemak Jenuh',
        'Lemak Trans',
        'Lemak tak Jenuh Ganda',
        'Lemak tak Jenuh Tunggal',
        'Serat',
        'Gula',
        'Kolesterol',
        'Sodium',
        'Kalium']
    
    dict_sub_macro = {}
    idx_mg = 0
    idx_g = 0
    # check left text
    merge = [x.text.strip() for x in (sub_left + left)]
    missing_values = set(sub_macro_) - set(merge)
    for key in missing_values:
        dict_sub_macro[key] = 0
        
    for idx, right_text in enumerate(rigth):
        portion = right_text.text.strip()
        if "kkal" in portion:
            dict_sub_macro["Kalori"] = portion
        elif "mg" in portion:
            dict_sub_macro[left[idx_mg].text.strip()] = portion
            idx_mg += 1
        else:
            dict_sub_macro[sub_left[idx_g].text.strip()] = portion
            idx_g += 1
    return dict_sub_macro

In [7]:
def get_information_recipes(links):
    dict_information = {
        'Name': [],
        'Meal Type': [],
        'Serving': [],
        'Prep Time': [],
        'Cook Time': [],
        'Ingredients': [],
        'Intructions': [],
        'Energi': [],
        'Kalori': [],
        'Lemak': [],
        'Protein': [],
        'Karbohidrat': [],
        'Lemak Jenuh': [],
        'Lemak Trans': [],
        'Lemak tak Jenuh Ganda': [],
        'Lemak tak Jenuh Tunggal': [],
        'Kolesterol': [],
        'Serat': [],
        'Gula': [],
        'Sodium': [],
        'Kalium': [],
    }
    idx = 0
    for link in links:
        print(f'{idx}. https://www.fatsecret.co.id{link}')
        url_get_content = requests.get(f'https://www.fatsecret.co.id{link}')
        soup_content = BeautifulSoup(url_get_content.content, "html.parser")

        # ==GET RECIPES==
        # get name
        dict_information["Name"].append(soup_content.find("h1", class_="fn").text.strip())

        # get meal type
        meal_type = []
        meal_types = soup_content.find("div", id="mealtypes")
        for i in meal_types.find_all("div", class_="tag"):
            meal_type.append(i.text.strip())

        dict_information["Meal Type"].append(meal_type)

        # get portion
        serving = soup_content.find("div", id="servings")

        dict_information["Serving"].append(serving.find("div", class_="yield").text.strip())

        # get cooking time
        cooking_time = soup_content.find("div", id="cooktime")
        try:
            dict_information["Prep Time"].append(cooking_time.find("div", class_="prepTime").text.strip())
        except KeyError:
            dict_information["Prep Time"].append('0 menit')
            
        try:
            # Coba mengambil nilai dari atribut "text"
            cook_time_element = cooking_time.find("div", class_="cookTime")
            if cook_time_element is not None:
                dict_information["Cook Time"].append(cook_time_element.text.strip())
            else:
                dict_information["Cook Time"].append('0 menit')
        except AttributeError:
            dict_information["Cook Time"].append('0 menit')


        # get ingredients
        ingredients = []
        for i in soup_content.find_all("li", class_="ingredient"):
            ingredients.append(i.text.strip())
            
        dict_information["Ingredients"].append(ingredients)

        # get intructions
        intructions = []
        for i in soup_content.find_all("li", class_="instruction"):
            intructions.append(i.text.strip())

        dict_information["Intructions"].append(intructions)

        # ===GET NUTRITION==
        nutrition_fact = soup_content.find("div", class_="nutrition_facts international")

        # get macro
        macro_left = nutrition_fact.find_all("div", class_="nutrient black left")
        macro_right = nutrition_fact.find_all("div", class_="nutrient black right tRight")
        macro = get_macro(macro_left, macro_right)

        for key, value in macro.items():
            dict_information[key].append(value)

        # get sub macro
        right_texts = nutrition_fact.find_all("div", class_="nutrient right tRight")
        sub_left_texts = nutrition_fact.find_all("div", class_="nutrient sub left")
        left_texts = nutrition_fact.find_all("div", class_="nutrient left")
        sub_macro = get_sub_macro(left_texts, sub_left_texts, right_texts)
        
        for key, value in sub_macro.items():
            try:
                dict_information[key].append(value)
            except KeyError as e:
                print(f"KeyError: {e}")
                pass
        idx += 1
    print("Scraping Has Been Completed..")            
    return dict_information

In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
all_link_recipes = []
LEN_ALL_JOB_PAGE = 17

for i in range(LEN_ALL_JOB_PAGE):
    if i == 0:
        url_get = requests.get(f'https://www.fatsecret.co.id/Default.aspx?pa=rs')
    else:
        url_get = requests.get(f'https://www.fatsecret.co.id/Default.aspx?pa=rs&pg={i}')
    soup_all = BeautifulSoup(url_get.content, "html.parser")
    containers = soup_all.find_all("form", id="ctl03")[0]
    recipes = containers.find("table", class_="listtable searchResult")
    links = recipes.find_all("a", class_="prominent")
    for link in links:
        all_link_recipes.append(link.get("href"))
print("All links have been obtained..")
    

All links have been obtained..


In [10]:
len(all_link_recipes)

327

In [11]:
data = get_information_recipes(all_link_recipes)

0. https://www.fatsecret.co.id/resep/74332855-donat/Default.aspx


1. https://www.fatsecret.co.id/resep/71694317-muffin-oat-pisang/Default.aspx
2. https://www.fatsecret.co.id/resep/70672846-banana-bread-oats/Default.aspx
3. https://www.fatsecret.co.id/resep/70171367-bilah-energi-oatmeal-cokelat/Default.aspx
4. https://www.fatsecret.co.id/resep/67444129-kue-pisang/Default.aspx
5. https://www.fatsecret.co.id/resep/67236434-mie-shirataki-goreng/Default.aspx
6. https://www.fatsecret.co.id/resep/66836366-kue-oat-pisang/Default.aspx
7. https://www.fatsecret.co.id/resep/66824678-gandum-dengan-buah/Default.aspx
8. https://www.fatsecret.co.id/resep/66790743-saus-cokelat/Default.aspx
9. https://www.fatsecret.co.id/resep/66245292-sandwich-telur-dadar/Default.aspx
10. https://www.fatsecret.co.id/resep/65593665-banana-oats-coklat/Default.aspx
11. https://www.fatsecret.co.id/resep/65370350-wafel/Default.aspx
12. https://www.fatsecret.co.id/resep/65355205-tumis-kubis/Default.aspx
13. https://www.fatsecret.co.id/resep/65329105-jus-hijau/Default.aspx
14. https://www.f

In [12]:
import pandas as pd
dataframe = pd.DataFrame(data)

In [13]:
dataframe

,Name,Meal Type,Serving,Prep Time,Cook Time,Ingredients,Intructions,Energi,Lemak,Protein,...,Kalori,Lemak Jenuh,Lemak Trans,Lemak tak Jenuh Ganda,Lemak tak Jenuh Tunggal,Kolesterol,Serat,Gula,Sodium,Kalium
0,Donat,"[Roti & Produk Panggang, Makanan Ringan, Makan...",12 porsi,20 menit,15 menit,"[3 gram ragi, 40 gram kuning telur, 30 gram gu...","[Campur semua bahan kering., Tambahkan semua b...",444 kj,"1,73g","3,7g",...,106 kkal,"0,634g",0,"0,205g","0,528g",43mg,"0,1g","3,46g",10mg,26mg
1,Muffin Oat Pisang,"[Roti & Produk Panggang, Makanan Penutup, Maka...",5 porsi,10 menit,25 menit,"[1 sedang telur, 1 sdt ekstrak vanila, 154 gra...","[Masukan semua bahan pada blender., Setelah te...",571 kj,"2,98g","4,02g",...,136 kkal,"0,593g",0g,"0,714g","1,203g",37mg,"2,5g","5,93g",295mg,138mg
2,Banana Bread Oats,"[Makanan Ringan, Makanan Penutup, Roti & Produ...",4 porsi,15 menit,15 menit,"[2 besar telur, 1 sdt kayu manis, 115 g banana...",[Panaskan oven/airfryer dengan suhu 160°C sela...,1032 kj,"8,53g","8,84g",...,247 kkal,"1,231g",0g,"1,202g","2,24g",106mg,"4,4g","7,64g",76mg,142mg
3,Bilah Energi Oatmeal Cokelat,"[Makanan Ringan, Makanan Penutup]",16 porsi,30 menit,0 menit,[25 gram biji bunga matahari (yang) dikuliti s...,[Sangrai oatmeal sampai kecoklatan. Dinginkan ...,386 kj,"3,61g","2,06g",...,92 kkal,"0,552g",0g,"0,674g","0,572g",0mg,1g,"7,44g",26mg,71mg
4,Kue Pisang,"[Makanan Penutup, Makanan Ringan, Lainnya, Rot...",4 porsi,15 menit,20 menit,"[1 besar telur, 1 sdt kayu manis, 1/4 sdt gara...","[Hancurkan pisang., Aduk telur sampai berbusa,...",492 kj,"2,76g","6,3g",...,118 kkal,"0,674g",0g,"0,559g","1,026g",57mg,"2,4g","4,88g",492mg,156mg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Tumis Brokoli Udang,"[Hidangan Utama, Makan Siang]",1 porsi,5 menit,10 menit,"[100 gram udang, 2 mangkok, dicincang brokoli,...",[Tumis bawang putih menggunakan minyak zaitun ...,1377 kj,"16,54g","32,91g",...,329 kkal,"2,384g",0,"2,406g","10,214g",206mg,"4,8g","3,12g",674mg,763mg
323,Tumis Oatmeal Dengan Sayur,[Makan Pagi],1 porsi,10 menit,15 menit,"[1 sejumput lada, 1 sdt garam, 1 mangkok, dici...","[Sangrai oatmeal, kurang lebih 5 menit, sampai...",1142 kj,"0,5g","7,1g",...,273 kkal,"0,08g",0g,"0,242g","0,026g",0mg,"19,9g","7,15g",2441mg,605mg
324,Sup Labu Energi,"[Sup, Makan Pagi]",1 porsi,10 menit,30 menit,"[1 gelas susu tanpa lemak, 500 gram labu kunin...",[Potong labu hingga berbentuk bulan sabit sepe...,1042 kj,"0,8g","14,29g",...,249 kkal,"0,563g",0,"0,083g","0,189g",5mg,"2,8g","19,3g",108mg,2093mg
325,Tumis Kacang Panjang,"[Hidangan Utama, Makan Siang]",1 porsi,5 menit,10 menit,"[100 gram dada ayam, 55 g kacang panjang, 25 g...","[Potong kacang panjang, dada ayam, bawang bomb...",1172 kj,"13,31g","31,12g",...,280 kkal,"3,012g",0,"3,874g","5,232g",83mg,"2,4g","1,78g",468mg,411mg


In [15]:
dataframe.iloc[0]

Name                                                                   Donat
Meal Type                  [Roti & Produk Panggang, Makanan Ringan, Makan...
Serving                                                             12 porsi
Prep Time                                                           20 menit
Cook Time                                                           15 menit
Ingredients                [3 gram ragi, 40 gram kuning telur, 30 gram gu...
Intructions                [Campur semua bahan kering., Tambahkan semua b...
Energi                                                                444 kj
Lemak                                                                  1,73g
Protein                                                                 3,7g
Karbohidrat                                                            19,3g
Kalori                                                              106 kkal
Lemak Jenuh                                                           0,634g

In [14]:
file_name = "recipes_fatsecret.csv"

dataframe.to_csv(file_name, index=False)
print(f"Data has been saved in file '{file_name}'")

Data has been saved in file 'recipes_fatsecret.csv'
